In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sb
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.optimizers import SGD,Adam
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                                directory=r"./train/",
                                target_size=(48,48),
                                batch_size=32,
                                color_mode="grayscale",
                                class_mode='categorical')
test_generator = test_datagen.flow_from_directory(
                                directory=r"./test/",
                                target_size=(48,48),
                                batch_size=32,
                                color_mode="grayscale",
                                class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [3]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [78]:
model_log = model.fit_generator(train_generator,
                                steps_per_epoch=28709 // 64,
                                epochs=10,
                                validation_data=test_generator,
                                validation_steps=7178 // 64 )

Epoch 1/10
448/448 [==============================] - 172s 383ms/step - loss: 0.4080 - accuracy: 0.8469 - val_loss: 1.3350 - val_accuracy: 0.5837
Epoch 2/10
448/448 [==============================] - 178s 397ms/step - loss: 0.4330 - accuracy: 0.8405 - val_loss: 1.1256 - val_accuracy: 0.5960
Epoch 3/10
448/448 [==============================] - 165s 369ms/step - loss: 0.3858 - accuracy: 0.8554 - val_loss: 1.1053 - val_accuracy: 0.5811
Epoch 4/10
448/448 [==============================] - 158s 353ms/step - loss: 0.4329 - accuracy: 0.8416 - val_loss: 2.0810 - val_accuracy: 0.5893
Epoch 5/10
448/448 [==============================] - 167s 373ms/step - loss: 0.3812 - accuracy: 0.8639 - val_loss: 1.1590 - val_accuracy: 0.5870
Epoch 6/10
448/448 [==============================] - 164s 366ms/step - loss: 0.4270 - accuracy: 0.8457 - val_loss: 2.2137 - val_accuracy: 0.5790
Epoch 7/10
448/448 [==============================] - 160s 357ms/step - loss: 0.3746 - accuracy: 0.8632 - val_loss: 2.1088 -

In [3]:
import cv2

In [6]:
img = cv2.imread('aman.jpg',0)
img=cv2.resize(img,(48,48))
img= np.expand_dims(img, axis =0)
img = np.expand_dims(img, axis=3)
img= img/255.0

In [ ]:
predictions=model.predict(img)
max_index=np.argmax(predictions[0])
emotions = ('angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise')
predicted_emotion = emotions[max_index]
print(predicted_emotion)

In [79]:
emotion_json = model.to_json()
with open("emotion.json", "w") as json_file:
    json_file.write(emotion_json)
model.save_weights("emotion.h5")